<a href="https://colab.research.google.com/github/drew-walkerr/DetectingBias/blob/main/detecting_bias_neural_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial Links from class
1. https://www.kaggle.com/deshwalmahesh/nlp-beginner-1-rnn-lstm-gru-embeddings-glove
2. https://www.kaggle.com/kredy10/simple-lstm-for-text-classification 

3. https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/


In [24]:
# This is a BoW classifier to detect the presence of provider scare quotes in provider clinical notes

import pandas as pd
import numpy
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn import feature_selection
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf

tfds.disable_progress_bar()


In [25]:
import matplotlib.pyplot as plt


def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [26]:
# Load in data
gold_standard = pd.read_csv("gold_standard_bias_annotation_doc_training.csv")


Mini HW 9/10—Neural classifier —Instructions:
Due 10 Dec
In this two part HW, you will split your dataset into training/test sets, and then build a neu-
ral classifier of your choice. Recommend LSTM but for an extra challenge you can also consider a transformer/RoBERTA if you prefer (esp if you are working with a low resource language). Do this homework in python only. You will need to move to google co-lab if you haven’t already and take
advantage of a GPU.

1. First, for HW 8, set up your neural model. You will likely want to use the same training/test split
as before, but if you make any adjustments make sure to note the reason. You will need to include
Glove embeddings as a feature of your LSTM model. Consider the many available tutorials such
as this or this. Consider at least two specifications of your neural model, and write up what you
did and why.





In [7]:
# Split
seed = 1234
X = gold_standard['Sentence']
y = gold_standard['quote_use']
#Hyperparameters
max_features_model = 3000
splits = 5
model_name = "RNN"
#Splitting and model
skf = StratifiedKFold(n_splits=splits, random_state=seed, shuffle= True)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

max_words = 10000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)





In [8]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [9]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           500000    
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [10]:
model.fit(sequences_matrix,y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
3/3 [==============================] - 4s 473ms/step - loss: 0.6468 - accuracy: 0.2032 - val_loss: 0.4403 - val_accuracy: 0.2660
Epoch 2/10
3/3 [==============================] - 1s 217ms/step - loss: -0.1544 - accuracy: 0.1658 - val_loss: -2.3928 - val_accuracy: 0.2660


In [19]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
y_pred_class = model.predict(test_sequences_matrix)

In [12]:
accr = model.evaluate(test_sequences_matrix,y_test)


4/4 [==============================] - 0s 21ms/step - loss: -1.7684 - accuracy: 0.1880


In [18]:
print(accr)
print(metrics.classification_report(y_test, test_sequences_matrix, digits=5))



[-1.7683783769607544, 0.18803419172763824]


ValueError: ignored

In [13]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: -1.768
  Accuracy: 0.188


2. Next, for HW 10, estimate the F1 score and plot a precision/recall curve to compare your neural
model specifications to your BOW model in the same plot. Consider using a tutorial (e.g. here).
Write up in overleaf what you did and why, and include the plot that compares your neural and
BOW models in your write up on overleaf and submit on Canvas.
